In [1]:
dataset_id = "college2_staging"

In [3]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'cs327e-fa2019:college2_staging' already
exists.


## Loading CSV files from Cloud Storage

In [4]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.Current_Student \
'gs://college_dataset/raw/Current_Students.csv'

Waiting on bqjob_r184ea3d9dc54b174_0000016e5d1d98e6_1 ... (1s) Current status: DONE   


In [6]:
%%bigquery 
select count(*) from college2_staging.Current_Student

,f0_
0,11


In [7]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.New_Student \
'gs://college_dataset/raw/New_Students.csv'

Waiting on bqjob_r1a7c5418ef84302b_0000016e5d202629_1 ... (2s) Current status: DONE   


In [8]:
%%bigquery 
select count(*) from college2_staging.New_Student

,f0_
0,7


In [9]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.Class \
'gs://college_dataset/raw/Classes.csv'

Waiting on bqjob_r1705ae61e98cd983_0000016e5d20cf7c_1 ... (2s) Current status: DONE   


In [10]:
%%bigquery 
select count(*) from college2_staging.Class

,f0_
0,12


## Creating Modeled Tables 

In [11]:
modeled_dataset_id = "college2_modeled"

In [12]:
!bq --location=US mk --dataset {modeled_dataset_id}

Dataset 'cs327e-fa2019:college2_modeled' successfully created.


In [18]:
%%bigquery
create or replace table college2_modeled.Student as
select distinct sid, fname, lname, dob
from college2_staging.Current_Student
union distinct
select distinct sid, fname, lname, cast(dob as string) as dob
from college2_staging.New_Student

""


In [19]:
%%bigquery
select count(*) from college2_modeled.Student

,f0_
0,15


In [20]:
%%bigquery
create or replace table college2_modeled.Teacher as
select distinct tid, instructor, dept
from college2_staging.Class

""


In [21]:
%%bigquery
select count(*) from college2_modeled.Teacher

,f0_
0,12


In [22]:
%%bigquery
select * from college2_modeled.Teacher limit 10

,tid,instructor,dept
0,cannata,PHILIP CANNATA,CS
1,None,Prof. Glenn Downing,CS
2,None,Prof. Alison Norman,CS
3,mitra,"Mitra, Shyamal",CS
4,cannata,"Cannata, Philip",CS
5,koch,"Koch, Hans",Math
6,mueller,"MUELLER, PETER",Math
7,neeman,JOE NEEMAN,Mathematics
8,tran,"Tran, Ngoc",Mathematics
9,bulko,bill bulko,Computer Science


In [23]:
%%bigquery
create or replace table college2_modeled.Takes as
select distinct sid, cno, grade
from college2_staging.Current_Student
where sid is not null
and cno is not null

""


In [24]:
%%bigquery
select * from college2_modeled.Takes limit 10

,sid,cno,grade
0,jc,CS327E,None
1,kev18,CS329E,None
2,paulg,CS329E,A
3,paulg,CS326E - Fall18,A-
4,paulg,CS313E,None
5,bzen26,CS313E - Fall18,B+
6,jerryh,CS329E,A-
7,jerryh,CS327E,B
8,jerryh,CS327E,None


In [26]:
%%bigquery 
create table college2_modeled.Teaches as
select distinct tid, cno
from college2_staging.Class
where tid is not null
and cno is not null

""


In [27]:
%%bigquery
select * from college2_modeled.Teaches limit 10

,tid,cno
0,koch,M328K
1,tran,M358K
2,bulko,CS313E
3,mitra,CS329E
4,mitra,CS313E
5,neeman,M362K
6,scohen,CS327E
7,cannata,CS347
8,cannata,CS326E
9,mueller,M362K
